In [4]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import jarque_bera, norm, t, chisquare
from statsmodels.stats.power import tt_ind_solve_power

%matplotlib inline


In [5]:
start_array, stop_array, step_array = 10, 100, 10
start_prob, stop_prob,  step_prob = 0.5, 1.0, 0.01
fisrt_norm, second_norm, student = 0, 3, 3
vyborka = []
params = []
count_size = 1
array = np.arange(start_array, stop_array+step_array, step_array)
prob = np.arange(start_prob, stop_prob + step_prob, step_prob)



In [6]:
# generating distribution for every pair of params
for i in (array):
    for j in (prob):
        #TODO: count_size should be changed later - now it's too long to genereta so much samples
        for count in range(count_size):
            rand = np.random.uniform(start_prob, stop_prob)
            if rand > j:
                distr = j*norm(fisrt_norm, second_norm).rvs(i)
            else:
                distr = (1 - j)*t.rvs(student, i)
            vyborka.append(distr)
            params.append(i)

In [7]:
print(vyborka) #just checking, can be deleted later

[array([ 2.07795506,  1.96949325, -2.07102014,  0.03257151, -0.7841341 ,
       -0.79740084,  0.34030304,  0.10456071,  0.06114085,  0.38501377]), array([-1.80368515,  3.82210604, -0.81422185,  2.74144195,  0.09574316,
       -0.429181  , -0.41940534, -1.27809451, -1.7159983 , -1.16637311]), array([ 0.05337171, -1.93140271,  0.83249411, -2.55109889, -0.75573416,
       -2.01163832,  0.94683877,  0.58700396,  0.61555271, -0.66728857]), array([-0.09637718, -1.61507597, -0.69308518,  0.87045685,  1.36517668,
        0.80584176, -5.4279819 ,  2.51659915,  0.74240107, -1.93152488]), array([ 1.24170234,  0.80846004,  0.32314602, -0.40119428, -0.15862189,
       -1.9488815 ,  0.70554063,  0.55919142,  1.64379062, -2.58706043]), array([ 0.78205079,  3.08952675, -0.4769343 ,  0.8021873 , -1.01704588,
        0.77220095,  0.99258297,  2.8062287 , -1.48279202, -2.00148458]), array([-0.42714436, -2.96548987, -0.42631055,  1.80549461, -0.10567372,
        0.73003227,  1.01725095,  0.1983434 , -1.26